In [1]:
from util import load_eidc_data as eidc

df = eidc.load_as_df('data/catalogue_metadata.json')
df

,title,description,lineage
0,"Land Cover Map 2007 (25m raster, NI)",LCM2007 is a parcel-based thematic classificat...,LCM2007 uses a spatial framework based on OSNI...
1,Land Cover Map 2021 (25m rasterised land parce...,This is a 25m pixel data set representing the ...,UKCEH’s automated land cover algorithms classi...
2,"Land Cover Map 2000 (vector, GB)",This dataset consists of the vector version of...,"In a major development of earlier methods, LCM..."
3,"Land Cover Map 2021 (10m classified pixels, N....",This is a 10m pixel data set representing the ...,UKCEH’s automated land cover algorithms classi...
4,Telomere length data from a wild population of...,These data consist of relative telomere length...,Blood samples (~25 μl) were collected and stor...
...,...,...,...
1862,Potential evapotranspiration derived from HadU...,Gridded potential evapotranspiration calculate...,This dataset was derived from the HadUK-Grid g...
1863,Fine root production in human-modified forests...,This dataset includes measurements of soil res...,We assessed roots production in 20 study plots...
1864,Nitrous oxide fluxes and associated soil measu...,This data set includes over 500 individual flu...,Flux measurements were made using a high-preci...
1865,A land use map of Peninsular Malaysia for the ...,Gridded land use map of Peninsular Malaysia wi...,The land use is based on detailed polygon land...


In [34]:
from transformers import pipeline

model_names = ['deepset/roberta-base-squad2', 'distilbert/distilbert-base-cased-distilled-squad', 'deepset/minilm-uncased-squad2']
qa_pipes = {}
for model_name in model_names:
    qa_pipes[model_name] = pipeline('question-answering', model=model_name)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
dataset = df.loc[df['title'] == 'Location data of worker bumblebees across an agricultural landscape in Buckinghamshire, UK']
context = dataset.iloc[0]['description']
context

'This dataset contains locations of worker bumblebees of five species (Bombus terrestris, B. lapidarius, B. pascuorum, B. hortorum, B. ruderatus) across an agricultural landscape centred on the Hillesden Estate, Buckinghamshire, UK. Locations were recorded in the field using a handheld GPS unit. Workers were non-lethally DNA sampled between June and August 2011, and genetic analysis used to confirm species and assign individuals to full-sib groups (colonies). Data were collected as part of a project led by the Centre for Ecology & Hydrology, funded under the Insect Pollinators Initiative.'

In [36]:
questions = ['How was DNA sampling performed?', 'Who funded the project?', 'Where was the data gathered?']
for q in questions:
    print(q)
    for model_name in model_names:
        print(model_name)
        ans = qa_pipes[model_name](q, context)
        print(ans)

How was DNA sampling performed?
{'score': 0.12974794209003448, 'start': 309, 'end': 321, 'answer': 'non-lethally'}
{'score': 0.25504252314567566, 'start': 277, 'end': 294, 'answer': 'handheld GPS unit'}
{'score': 0.8052693605422974, 'start': 309, 'end': 321, 'answer': 'non-lethally'}
Who funded the project?
{'score': 0.2713090181350708, 'start': 515, 'end': 545, 'answer': 'Centre for Ecology & Hydrology'}
{'score': 0.20751163363456726, 'start': 564, 'end': 593, 'answer': 'Insect Pollinators Initiative'}
{'score': 0.6175491213798523, 'start': 564, 'end': 593, 'answer': 'Insect Pollinators Initiative'}
Where was the data gathered?
{'score': 0.16213250160217285, 'start': 193, 'end': 230, 'answer': 'Hillesden Estate, Buckinghamshire, UK'}
{'score': 0.544015109539032, 'start': 193, 'end': 230, 'answer': 'Hillesden Estate, Buckinghamshire, UK'}
{'score': 0.24066263437271118, 'start': 193, 'end': 230, 'answer': 'Hillesden Estate, Buckinghamshire, UK'}
